## API integrating the EBRAINS KG with TVB using `siibra-python`.
### Focuses on retrieving and creating TVB Structural and Functional Connectivities

In [ ]:
import siibra

## Using the API

## 1. Retrieving *both* Structural and Functional Connectivities

##### The results of this method are 2 **dictionaries**, containing structural and functional connectivities respectively. Each dictionary has as:
##### **- key**: the subject id
##### **- value**: the Structural/Functional TVB Connectivity for the corresponding subject

In [ ]:
from tvb.adapters.creators.siibra_base import get_connectivities_from_kg
struct_conn_dict, func_conn_dict = get_connectivities_from_kg('human', 'julich 3', 'HCP', '000-001', True)

##### There is **1** Structural connectivity for each subject:

In [ ]:
print(f'Structural connectivities:')
for sc in struct_conn_dict.items():
    print(sc)

##### And **5** Functional connectivities for each subject:

In [ ]:
print(f'Functional connectivities:')
for fc in func_conn_dict.items():
    print(fc)

#### After retrieving the connectivities, we can access and use them as any other connectivity from TVB:

In [ ]:
sc_conn = struct_conn_dict['000']
sc_conn

In [ ]:
from tvb.simulator.plot.tools import plot_connectivity
plot_connectivity(sc_conn)

---

## 2. Retrieving *just* Structural Connectivities

#### 2.1 Using the common API for Structural and Functional Connectivities:
####     The API from 1. can be used to extract just Structural connectivities, by setting the **last flag** (mentioning the option to also compute Functional connectivities) to **False**

In [ ]:
struct_conn_dict, func_conn_dict = get_connectivities_from_kg('human', 'julich 2.9', '1000BRAINS', '0002', False)

##### Now there are 2* Structural connectivities for our subject:
##### **in this cohort, some subjects had 2 scanning sessions, resulting in 2 Structural connectivities*

In [ ]:
print(f'Structural connectivities:')
for sc in struct_conn_dict.items():
    print(sc)

##### And no Functional connectivities:

In [ ]:
print(f'Functional connectivities: \n {func_conn_dict}')

#### 2.2 Using the custom API for Structural connectivities:

In [ ]:
from tvb.adapters.creators.siibra_base import get_structural_connectivities_from_kg
struct_conn_dict = get_structural_connectivities_from_kg('human', '2.9', '1000BRAINS', '0002')

In [ ]:
print(f'Structural connectivities:')
for sc in struct_conn_dict.items():
    print(sc)

#### Again, the resulted connectivity could be used like any other TVB Connectivity

---

## 3. Retrieving *just* Functional Connectivities
#### Note: Functional Connectivities are represented as ConnectivityMeasures in TVB. This means that each FC will have an associated Structural Connectivity to it. 
#### For this reason, to extract any FC from siibra, we must also provide a dictionary of corresponding Structural Connectivities.

In [ ]:
struct_conn_dict = get_structural_connectivities_from_kg('human', 'julich 3', 'HCP', '002')

In [ ]:
from tvb.adapters.creators.siibra_base import get_connectivity_measures_from_kg
func_conn_dict = get_connectivity_measures_from_kg('human', '2.9', 'HCP', '002', struct_conn_dict)

In [ ]:
print(f'Functional connectivities:')
for fc in func_conn_dict.items():
    print(fc)

In [ ]:
# inspecting a Functional Connectivity
func_conn_dict['002'][0]